In [6]:
import numpy as np
import scipy as sp
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import svm, grid_search, datasets
import math
import seaborn as sns
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from texttable import Texttable
import texttable as tt
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from statistics import mean
import sys, os


from scipy.stats import boxcox
#import lightgbm as lgb

from sklearn.datasets import dump_svmlight_file
from sklearn.externals import joblib
from sklearn.metrics import precision_score

from sklearn.metrics import accuracy_score
from xgboost import plot_importance
import sklearn.cluster as cluster
import time
from collections import Counter
from sklearn.model_selection import cross_val_predict
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.optimize import curve_fit
import hdbscan
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns
import array
from sklearn import metrics
pd.options.mode.chained_assignment = None
import pandas_profiling

from plotly.offline import iplot,init_notebook_mode, plot as pplot
from plotly import graph_objs as go
init_notebook_mode()
import plotly.plotly as py
py.sign_in('emaad', 'VzbDk0qpBaRDSJevZ3bo')
from plotly import tools

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [7]:
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

In [8]:
class MLCollisions(object):
    def __init__(self, dataframe, metric2opt, FI=False):
        
        maindataframe = dataframe
        
        maindataframe = maindataframe[maindataframe['Mass of Remnant'] != 0] #Removing Zeroes
                
        maindataframe["Impactor Mass"] = maindataframe["Impactor Mass/Target Mass"]*maindataframe["Target Mass"] #Getting Mass of Impactor

        maindataframe["Xi"] = (maindataframe["Mass of Remnant"] - maindataframe["Target Mass"])/(maindataframe["Impactor Mass/Target Mass"]*maindataframe["Target Mass"]) #Calculating Xi (efficiency)
        maindataframe["RemTarg"] = maindataframe["Mass of Remnant"]/maindataframe["Target Mass"]
        maindataframe["RemTot"] = maindataframe["Mass of Remnant"]/(maindataframe["Target Mass"]+maindataframe["Impactor Mass"])

        #Classifying by Hand
        xilist = maindataframe["Xi"].tolist()
        tar = maindataframe["Target Mass"].tolist()
        imp = maindataframe["Impactor Mass"].tolist()
        
        #Classifying
        clsfr = []
        for i in range(0,len(xilist)):
            if (xilist[i] < tar[i] and xilist[i] >= 0.1*tar[i]):
                clsfr.append(0) #Erosion = MLR = 0.1-0.9MTarg - Xi below 0, above MTarg/Mimp
            elif (xilist[i] < (tar[i]+ 0.9*imp[i]) and xilist[i] > (tar[i]+ 0.1*imp[i])):
                clsfr.append(1) #Partial Accretion = MLR = MTarg + 0.1-0.9MImp - Xi in [0.1-0.9]
            elif (xilist[i] >= (tar[i]+ 0.9*imp[i])):
                clsfr.append(2) #Perfect merger = MLR = MTarg + MImp - Xi = 1
            else:
                clsfr.append(3) #Hit and run = MLR = MTarg - Xi = 0

        maindataframe["Group"] = clsfr

        group_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle']]
        group_y=maindataframe['Group']
        
        
        
        xi_wgroup_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle','Group']]
        xi_wgroup_y=maindataframe['Xi']
        
        xi_nogroup_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle']]
        xi_nogroup_y=maindataframe['Xi'] 
        
        
        
        remtarg_wgroup_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle','Group']]
        remtarg_wgroup_y=maindataframe['RemTarg']
        
        remtarg_nogroup_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle']]
        remtarg_nogroup_y=maindataframe['RemTarg'] 
        
        
        
        remtot_wgroup_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle','Group']]
        remtot_wgroup_y=maindataframe['RemTot']
        
        remtot_nogroup_X=maindataframe[['Target Mass', 'Impactor Mass/Target Mass', 'Impact Velocity', 'Impact Angle']]
        remtot_nogroup_y=maindataframe['RemTot'] 
        
        
        
        group_X_train, group_X_test, group_y_train, group_y_test = train_test_split(group_X, group_y, test_size=0.25, random_state=0)
        group_data_TRAIN=pd.concat([group_X_train, group_y_train], axis=1)
        group_data_TEST=pd.concat([group_X_test, group_y_test],axis=1)
        
        self.group_X_train = group_X_train
        self.group_y_train = group_y_train
        self.group_X_test = group_X_test
        self.group_y_test = group_y_test
        self.group_data_TRAIN = group_data_TRAIN
        self.group_data_TEST = group_data_TEST
        
        

        xi_wgroup_X_train, xi_wgroup_X_test, xi_wgroup_y_train, xi_wgroup_y_test = train_test_split(xi_wgroup_X, xi_wgroup_y, test_size=0.25, random_state=0)
        xi_wgroup_data_TRAIN=pd.concat([xi_wgroup_X_train, xi_wgroup_y_train], axis=1)
        xi_wgroup_data_TEST=pd.concat([xi_wgroup_X_test, xi_wgroup_y_test],axis=1) 
        
        xi_nogroup_X_train, xi_nogroup_X_test, xi_nogroup_y_train, xi_nogroup_y_test = train_test_split(xi_nogroup_X, xi_nogroup_y, test_size=0.25, random_state=0)
        xi_nogroup_data_TRAIN=pd.concat([xi_nogroup_X_train, xi_nogroup_y_train], axis=1)
        xi_nogroup_data_TEST=pd.concat([xi_nogroup_X_test, xi_nogroup_y_test],axis=1)
        
        self.xi_wgroup_X_train = xi_wgroup_X_train
        self.xi_wgroup_y_train = xi_wgroup_y_train
        self.xi_wgroup_X_test = xi_wgroup_X_test
        self.xi_wgroup_y_test = xi_wgroup_y_test
        self.xi_wgroup_data_TRAIN = xi_wgroup_data_TRAIN
        self.xi_wgroup_data_TEST = xi_wgroup_data_TEST
        
        self.xi_nogroup_X_train = xi_nogroup_X_train
        self.xi_nogroup_y_train = xi_nogroup_y_train
        self.xi_nogroup_X_test = xi_nogroup_X_test
        self.xi_nogroup_y_test = xi_nogroup_y_test
        self.xi_nogroup_data_TRAIN = xi_nogroup_data_TRAIN
        self.xi_nogroup_data_TEST = xi_nogroup_data_TEST
        
        
        
        remtarg_wgroup_X_train, remtarg_wgroup_X_test, remtarg_wgroup_y_train, remtarg_wgroup_y_test = train_test_split(remtarg_wgroup_X, remtarg_wgroup_y, test_size=0.25, random_state=0)
        remtarg_wgroup_data_TRAIN=pd.concat([remtarg_wgroup_X_train, remtarg_wgroup_y_train], axis=1)
        remtarg_wgroup_data_TEST=pd.concat([remtarg_wgroup_X_test, remtarg_wgroup_y_test],axis=1)
        
        remtarg_nogroup_X_train, remtarg_nogroup_X_test, remtarg_nogroup_y_train, remtarg_nogroup_y_test = train_test_split(remtarg_nogroup_X, remtarg_nogroup_y, test_size=0.25, random_state=0)
        remtarg_nogroup_data_TRAIN=pd.concat([remtarg_nogroup_X_train, remtarg_nogroup_y_train], axis=1)
        remtarg_nogroup_data_TEST=pd.concat([remtarg_nogroup_X_test, remtarg_nogroup_y_test],axis=1)
        
        self.remtarg_wgroup_X_train = remtarg_wgroup_X_train
        self.remtarg_wgroup_y_train = remtarg_wgroup_y_train
        self.remtarg_wgroup_X_test = remtarg_wgroup_X_test
        self.remtarg_wgroup_y_test = remtarg_wgroup_y_test
        self.remtarg_wgroup_data_TRAIN = remtarg_wgroup_data_TRAIN
        self.remtarg_wgroup_data_TEST = remtarg_wgroup_data_TEST
        
        self.remtarg_nogroup_X_train = remtarg_nogroup_X_train
        self.remtarg_nogroup_y_train = remtarg_nogroup_y_train
        self.remtarg_nogroup_X_test = remtarg_nogroup_X_test
        self.remtarg_nogroup_y_test = remtarg_nogroup_y_test
        self.remtarg_nogroup_data_TRAIN = remtarg_nogroup_data_TRAIN
        self.remtarg_nogroup_data_TEST = remtarg_nogroup_data_TEST
        
        
        
        remtot_wgroup_X_train, remtot_wgroup_X_test, remtot_wgroup_y_train, remtot_wgroup_y_test = train_test_split(remtot_wgroup_X, remtot_wgroup_y, test_size=0.25, random_state=0)
        remtot_wgroup_data_TRAIN=pd.concat([remtot_wgroup_X_train, remtot_wgroup_y_train], axis=1)
        remtot_wgroup_data_TEST=pd.concat([remtot_wgroup_X_test, remtot_wgroup_y_test],axis=1)
        
        remtot_nogroup_X_train, remtot_nogroup_X_test, remtot_nogroup_y_train, remtot_nogroup_y_test = train_test_split(remtot_nogroup_X, remtot_nogroup_y, test_size=0.25, random_state=0)
        remtot_nogroup_data_TRAIN=pd.concat([remtot_nogroup_X_train, remtot_nogroup_y_train], axis=1)
        remtot_nogroup_data_TEST=pd.concat([remtot_nogroup_X_test, remtot_nogroup_y_test],axis=1)
        
        self.remtot_wgroup_X_train = remtot_wgroup_X_train
        self.remtot_wgroup_y_train = remtot_wgroup_y_train
        self.remtot_wgroup_X_test = remtot_wgroup_X_test
        self.remtot_wgroup_y_test = remtot_wgroup_y_test
        self.remtot_wgroup_data_TRAIN = remtot_wgroup_data_TRAIN
        self.remtot_wgroup_data_TEST = remtot_wgroup_data_TEST
        
        self.remtot_nogroup_X_train = remtot_nogroup_X_train
        self.remtot_nogroup_y_train = remtot_nogroup_y_train
        self.remtot_nogroup_X_test = remtot_nogroup_X_test
        self.remtot_nogroup_y_test = remtot_nogroup_y_test
        self.remtot_nogroup_data_TRAIN = remtot_nogroup_data_TRAIN
        self.remtot_nogroup_data_TEST = remtot_nogroup_data_TEST
        
        maxdepth_range=np.arange(1,21,2)
        nestimators_range=np.arange(50,100,5)
        
        self.FI=FI
        
        self.metric2opt = metric2opt
        self.metric=self.metric2opt
        self.maxdepth_range = maxdepth_range
        self.nestimators_range = nestimators_range
        self.cv = KFold(n_splits=4, random_state=0) #random state fixed to get scores and prediction same in crossval
        

#------------------------------------------------------------------------------  

    def createfeatures(self,data):

        # LOGS of original 4 features
        data['log Target Mass']= np.log10(data['Target Mass'])
        data["Impactor Mass"] = data["Impactor Mass/Target Mass"]*data["Target Mass"] #Mass of the Impactor in Earth Masses
        data['log Impactor Mass']=np.log10(data['Impactor Mass'])
        data['log Impactor log Target'] = data['log Impactor Mass']/ data['log Target Mass']


        # NORMALIZING ML TARGET
        #data["Normalized MLR"] = data["Remnant Mass"]/data["Target Mass"] #Normalizing mass of largest remnant to total mass  
        #data["NormTot MLR"] = data["Remnant Mass"]/(data["Target Mass"]+data["Impactor Mass"]) #Normalizing mass of largest remnant to total mass
        #data['log Mass of Remnant']=np.log10(data['Remnant Mass'])
        #data["Xi"] = (data["Remnant Mass"] - data["Target Mass"])/data["Impactor Mass"] #Calculating Xi (efficiency)

        data["Mass Total"] = data["Impactor Mass"]+data["Target Mass"] #Total Mass in Earth Masses

        impact_angleMLR = data["Impact Angle"].tolist() #Making a list from the Impact Angle column

        data["Rci"] = (((data["Target Mass"]+data["Impactor Mass"])/(1333.33*np.pi)))**((1/3)) #Spherical radius of the combined projectile and target masses (normalized to total mass) at a density of ρ ≡ 1000 kg/m^3
        data["Target Radius"] = (((data["Target Mass"])/(1333.33*np.pi)))**((1/3)) #Radius of the target (normalized to total mass) at a density of ρ ≡ 1000 kg/m^3
        data["Impactor Radius"] = (((data["Impactor Mass"])/(1333.33*np.pi)))**((1/3)) #Radius of the impactor in (normalized to total mass) at a density of ρ ≡ 1000 kg/m^3

        vstarcoff1 = ((32*np.pi*1.9)/5)**0.5 #VStar Coefficient 1 to calculate V* (critical impact velocity)
        vstarcoff2 = ((1000*6.674*(10**-11)))**0.5 #VStar Coefficient 2 to calculate V* (critical impact velocity)
        data["vstarcoff3"] = ((750*np.pi)/data["Mass Total"])**(1/3) #VStar Coefficient 3 to calculate V* (critical impact velocity)
        data["Vstar"] = data["vstarcoff3"]*vstarcoff2*vstarcoff1 #Critical Impact Velocity required to disperse half of the total mass for a specific impact scenario

        data["QR"] = (0.5*data["Impact Velocity"]*data["Impact Velocity"])/data["Mass Total"] #Center of mass specific impact energy
        data["QRD"] = (0.5*(data["Vstar"]*data["Vstar"]))/data["Mass Total"] #Catastrophic disruption threshold

        sinangle = [] #Getting the sine of the Impact Angle
        for i in range(0,len(impact_angleMLR)):
            x = impact_angleMLR[i]
            rad = math.radians(x)
            b = math.sin(rad)
            sinangle.append(b)

        data["Impact Parameter"] = sinangle #Creating a column with the sine of the angle
        data["B"] = (data["Impactor Radius"]+data["Target Radius"])*data["Impact Parameter"] #Sum of Radius multiplied by sine of angle
        data["l"] = (data["Impactor Radius"]+data["Target Radius"]) - data["B"] #Length of the projectile impacting the target
        data["alpha"] = ((3*data["Impactor Radius"]*(data["l"]**2))-(data["l"]**3))/(4*(data["Impactor Radius"]**3)) #Mass fraction of the projectile estimated to be involved in the collision
        data["mualpha"] = (data["alpha"]*data["Impactor Mass"]*data["Target Mass"])/((data["alpha"]*data["Impactor Mass"])+data["Target Mass"]) #Reduced mass that takes into account the effect of impact angle on QRD
        data["QRmu"] = (data["QR"])/data["mualpha"] #The difference of impact energy between a head-on impact by a projectile of mass M1 and a head-on impact by a projectile of mass alphaM1

        data["Sum Radius"] = data["Impactor Radius"]+data["Target Radius"] #Sum of Radii as calculated above

        data["bcrit"] = data["Target Radius"]/(data["Target Radius"]+data["Impactor Radius"]) #Critical impact parameter which shows transition from non-grazing to grazing collision (if bcrit < b (Impact Parameter), it is a Grazing Collision, where less than half of the projectile interacts or hits the target)

        data["gamma"] = data["Impactor Mass"]/data["Target Mass"] #Calculating weight ratio of Umpactor Mass/Target Mass and naming it gamma

        data["VinfVesc"] = (((data["Impact Velocity"])**2)-1)**0.5 #Calculating Vinf/Vesc based on the fact that Impact Velocity is Vimpact/Vescape, and Vinf/Vesc = SQRT((Vinf/Vesc)^2 - 1)

        data['vel x angle']=data['Impact Velocity']*data['Impact Angle']
        data['vel x angle2']=data['Impact Velocity']*data['Impact Angle']**2
        data['vel x angle3']=data['Impact Velocity']*data['Impact Angle']**3
        data['vel2 x angle']=data['Impact Velocity']**2*data['Impact Angle']
        data['vel3 x angle']=data['Impact Velocity']**3*data['Impact Angle']
        
        data['exp vel x angle']=np.exp(data['Impact Velocity'])*data['Impact Angle']
        data['sigmoid']= 1./(1.+ np.exp(data['Impact Velocity']))
        return data

    def outlierdetector(self,X_true,y_true,y_pred,errorbound):
        partdata = X_true.copy()
        partdata["Actual MLR"] = y_true
        partdata["Predicted MLR"] = y_pred
        partdata["Error"] = np.abs(partdata["Actual MLR"]-partdata["Predicted MLR"])
        baddata = partdata[partdata['Error'] > errorbound]
        gooddata = partdata[partdata['Error'] <= errorbound]

        print('Number of samples predicted poorly: %i' % len(baddata.index))
        print('MSE of poorly-predicted samples:  %f' % metrics.mean_squared_error(baddata["Predicted MLR"], baddata["Actual MLR"]))
        print('MSE of well-predicted samples:  %f' % metrics.mean_squared_error(gooddata["Predicted MLR"], gooddata["Actual MLR"]))

        #tracegood = go.Scatter(x=gooddata["Actual MLR"],y=gooddata["Predicted MLR"],name="Good Data", mode= 'markers',marker= dict(size= 3+3*gooddata['Impact Velocity'],line= dict(width=1), color= gooddata['Impact Angle'],colorscale='Blues',showscale=False))
        #tracebad = go.Scatter(x=baddata["Actual MLR"],y=baddata["Predicted MLR"],name="Bad Data", mode= 'markers',marker= dict(size= 3+3*baddata['Impact Velocity'],line= dict(width=1), color= baddata['Impact Angle'],colorscale='Reds',showscale=False))
        #traceact = go.Scatter(x=y_true,y=y_pred,name="All Data", mode= 'markers',marker= dict(size= 3+3*X_true['Impact Velocity'],line= dict(width=1), color= X_true['Impact Angle'],colorscale='Viridis',showscale=False))
        #traceline = go.Scatter(x = [min(y_true),max(y_true)],y = [min(y_true),max(y_true)],mode = 'lines',name = '1-1 Line',opacity=0.25)

        #dataplot = [tracegood,tracebad,traceact,traceline]
        #return py.iplot(dataplot)

    #def outlierdetectdata(self, X_true,y_true,y_pred,errorbound):
     #   partdata = X_true
      #  partdata["Actual MLR"] = y_true
       # partdata["Predicted MLR"] = y_pred
        #partdata["Error"] = np.abs(partdata["Actual MLR"]-partdata["Predicted MLR"])/np.abs(partdata["Actual MLR"])
        #baddata = partdata[partdata['Error'] > errorbound]
        #gooddata = partdata[partdata['Error'] <= errorbound]

        #return(gooddata, baddata)
    
    #-----XGBOOST CLASS SECTION--------#
    
    def xgbCV_scores(self, x_data, y_data, params):
        # cross validation with any params (needs to be a dictionary with max_depth and n_estimators)
        # eg. params={'max_depth': 5, 'n_estimators': 10}
        
        # instantiating model:
        xgbmodel=XGBRegressor(max_depth = params['max_depth'], 
                              n_estimators=params['n_estimators'], 
                              gamma=0.3, n_jobs=-1)
        
        # Cross validation for prediction and scores with same seed
        predictedCV = cross_val_predict(xgbmodel, x_data, y_data, cv=4)
        
        xgbmodel=XGBRegressor(max_depth = params['max_depth'], 
                              n_estimators=params['n_estimators'], 
                              gamma=0.3, n_jobs=-1)
        
        score=-cross_val_score(xgbmodel, x_data, y_data, scoring=self.metric, cv=4)
        score_mse=score
        if self.metric2opt != 'neg_mean_squared_error':
            xgbmodel=XGBRegressor(max_depth = params['max_depth'], 
                              n_estimators=params['n_estimators'], 
                              gamma=0.3, n_jobs=-1)
            score_mse=-cross_val_score(xgbmodel, x_data, y_data, scoring='neg_mean_squared_error', cv=4)
            
        return score, score_mse, predictedCV

    def xgbGS(self,mltarget):
        param_grid = {
                      'max_depth': self.maxdepth_range,
                      'n_estimators': self.nestimators_range
                      }

        # Initialize XGB and GridSearch
        xgb = XGBRegressor(gamma=0.3, njobs=-1) 
        grid = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring=self.metric, cv=4)
        print('Performing GridSearchCV with Metric:', self.metric2opt)
        if mltarget == 'xi':
            trainingset_X = self.xi_nogroup_X_train
            trainingset_y = self.xi_nogroup_y_train
        elif mltarget == 'remtot':
            trainingset_X = self.remtot_nogroup_X_train
            trainingset_y = self.remtot_nogroup_y_train
        elif mltarget == 'remtarg':
            trainingset_X = self.remtarg_nogroup_X_train
            trainingset_y = self.remtarg_nogroup_y_train
        elif mltarget == 'xifull':
            trainingset_X = self.xi_nogroup_full_X_train
            trainingset_y = self.xi_nogroup_y_train
        elif mltarget == 'remtotfull':
            trainingset_X = self.remtot_nogroup_full_X_train
            trainingset_y = self.remtot_nogroup_y_train
        elif mltarget == 'remtargfull':
            trainingset_X = self.remtarg_nogroup_full_X_train
            trainingset_y = self.remtarg_nogroup_y_train
        else:
            print('Error Found. Using Mass of Largest Remnant Normalized to Total Mass As Default')
            trainingset_X = self.remtot_nogroup_X_train
            trainingset_y = self.remtot_nogroup_y_train
            
        grid_result=grid.fit(trainingset_X, trainingset_y)

        #print('Best Estimator:',grid_result.best_estimator_)
        print('Best Parameters:', grid_result.best_params_)
        print('Best Metric Score:', -grid_result.best_score_)
        
        
        return grid_result.best_params_
    
    def xgbtrainmodel(self,X,y,params):
        # RETRAIN WITH ALL THE DATA:
        xgb = XGBRegressor(max_depth=params['max_depth'],
                         n_estimators=params['n_estimators'],  
                         gamma=0.3, n_jobs=-1)
        xgb_final = xgb.fit(X,y)
        features=X.columns.values.tolist()
        self.FeatureImportScores=pd.DataFrame({'feature': features,'score': xgb_final.feature_importances_,})
        
        if self.FI is True:
            print('----------')
            print('NON-ZERO FEATURE IMPORTANCES:')
            for i, f in enumerate(features):
                if xgb_final.feature_importances_[i] > 0:
                    print('%s: .... %2f' %(f, xgb_final.feature_importances_[i]))

        
        return xgb_final
    
    def xgbtest_unseendata(self, xgb_final,testX,testY):

        y_test_pred = xgb_final.predict(testX)

        print('Test Data Scores:')
        #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(testY, y_test_pred)))
        print('MAE : ....%2f' %(metrics.mean_absolute_error(testY, y_test_pred)))
        print('MSE : ....%2f' %(metrics.mean_squared_error(testY, y_test_pred)))
        print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(testY, y_test_pred))))
        return y_test_pred
    
    
    #-----NESTED CLASS SECTION--------#
    
    def nest_main(self,daataa,mltargetplanet):
        
        print("CLASS-SPECIFIC REGRESSOR TRAINING:")
        
        
        train_g0 = daataa[daataa["Group"] == 0]
        y_train_g0 = train_g0[mltargetplanet]
        del train_g0[mltargetplanet]
        X_train_g0 = train_g0
        
        model_g0 = XGBRegressor(gamma=0.3, n_jobs=-1) #XGBoost Model

        parameters_g0 = {'max_depth':self.maxdepth_range,'n_estimators':self.nestimators_range}
        xgbMLR_g0 = GridSearchCV(model_g0, parameters_g0)
        xgbMLR_g0.fit(X_train_g0,y_train_g0)
        bestparams_g0 = xgbMLR_g0.best_params_
        newxgb_g0 = XGBRegressor(max_depth = bestparams_g0.get('max_depth'), n_estimators = bestparams_g0.get('n_estimators'), gamma = 0.3, n_jobs=-1)
        newxgb_g0.fit(X_train_g0,y_train_g0)
        #predictedCV0 = cross_val_predict(newxgb_g0, X_train_g0, y_train_g0, cv=self.cv)
    
        score0i=-cross_val_score(newxgb_g0, X_train_g0, y_train_g0, scoring=self.metric2opt, cv=4)
            
        score_mse0i=score0i
            
        if self.metric2opt != 'neg_mean_squared_error':
            newxgb_g0 = XGBRegressor(max_depth = bestparams_g0.get('max_depth'), n_estimators = bestparams_g0.get('n_estimators'),gamma = 0.3, n_jobs=-1)
            score_mse0i=-cross_val_score(newxgb_g0, X_train_g0, y_train_g0, scoring='neg_mean_squared_error', cv=self.cv)
        
        print('Class 0 Training Data MSE: %2f .... STDev: %2f' %(np.mean(score_mse0i),np.std(score_mse0i)))
        print('Class 0 Training Data RMSE: %2f' %(np.sqrt(np.mean(score_mse0i))))
        print('')
        
        
        ######
        train_g1 = daataa[daataa["Group"] == 1]
        y_train_g1 = train_g1[mltargetplanet]
        del train_g1[mltargetplanet]
        X_train_g1 = train_g1
        
        model_g1 = XGBRegressor(gamma=0.3, n_jobs=-1) #XGBoost Model

        parameters_g1 = {'max_depth':self.maxdepth_range,'n_estimators':self.nestimators_range}
        xgbMLR_g1 = GridSearchCV(model_g1, parameters_g1)
        xgbMLR_g1.fit(X_train_g1,y_train_g1)
        bestparams_g1 = xgbMLR_g1.best_params_
        newxgb_g1 = XGBRegressor(max_depth = bestparams_g1.get('max_depth'), n_estimators = bestparams_g1.get('n_estimators'), gamma = 0.3, n_jobs=-1)
        newxgb_g1.fit(X_train_g1,y_train_g1)
        #predictedCV1 = cross_val_predict(newxgb_g1, X_train_g1, y_train_g1, cv=self.cv)
        
        score1i=-cross_val_score(newxgb_g1, X_train_g1, y_train_g1, scoring=self.metric2opt, cv=4)
            
        score_mse1i=score1i
            
        if self.metric2opt != 'neg_mean_squared_error':
            newxgb_g1 = XGBRegressor(max_depth = bestparams_g1.get('max_depth'), n_estimators = bestparams_g1.get('n_estimators'),gamma = 0.3, n_jobs=-1)
            score_mse1i=-cross_val_score(newxgb_g1, X_train_g1, y_train_g1, scoring='neg_mean_squared_error', cv=self.cv)
        
        print('Class 1 Training Data MSE: %2f .... STDev: %2f' %(np.mean(score_mse1i),np.std(score_mse1i)))
        print('Class 1 Training Data RMSE: %2f' %(np.sqrt(np.mean(score_mse1i))))
        print('')
        
        #######
        train_g2 = daataa[daataa["Group"] == 2]
        y_train_g2 = train_g2[mltargetplanet]
        del train_g2[mltargetplanet]
        X_train_g2 = train_g2
        
        model_g2 = XGBRegressor(gamma=0.3, n_jobs=-1) #XGBoost Model

        parameters_g2 = {'max_depth':self.maxdepth_range,'n_estimators':self.nestimators_range}
        xgbMLR_g2 = GridSearchCV(model_g2, parameters_g2)
        xgbMLR_g2.fit(X_train_g2,y_train_g2)
        bestparams_g2 = xgbMLR_g2.best_params_
        newxgb_g2 = XGBRegressor(max_depth = bestparams_g2.get('max_depth'), n_estimators = bestparams_g2.get('n_estimators'), gamma = 0.3, n_jobs=-1)
        newxgb_g2.fit(X_train_g2,y_train_g2)
        #predictedCV2 = cross_val_predict(newxgb_g2, X_train_g2, y_train_g2, cv=self.cv)
        
        score2i=-cross_val_score(newxgb_g2, X_train_g2, y_train_g2, scoring=self.metric2opt, cv=4)
            
        score_mse2i=score2i
            
        if self.metric2opt != 'neg_mean_squared_error':
            newxgb_g2 = XGBRegressor(max_depth = bestparams_g2.get('max_depth'), n_estimators = bestparams_g2.get('n_estimators'),gamma = 0.3, n_jobs=-1)
            score_mse2=-cross_val_score(newxgb_g2, X_train_g2, y_train_g2, scoring='neg_mean_squared_error', cv=self.cv)
        
        print('Class 2 Training Data MSE: %2f .... STDev: %2f' %(np.mean(score_mse2i),np.std(score_mse2i)))
        print('Class 2 Training Data RMSE: %2f' %(np.sqrt(np.mean(score_mse2i))))  
        print('')
        
        #######
        train_g3 = daataa[daataa["Group"] == 3]
        y_train_g3 = train_g3[mltargetplanet]
        del train_g3[mltargetplanet]
        X_train_g3 = train_g3
        
        model_g3 = XGBRegressor(gamma=0.3, n_jobs=-1) #XGBoost Model

        parameters_g3 = {'max_depth':self.maxdepth_range,'n_estimators':self.nestimators_range}
        xgbMLR_g3 = GridSearchCV(model_g3, parameters_g3)
        xgbMLR_g3.fit(X_train_g3,y_train_g3)
        bestparams_g3 = xgbMLR_g3.best_params_
        newxgb_g3 = XGBRegressor(max_depth = bestparams_g3.get('max_depth'), n_estimators = bestparams_g3.get('n_estimators'),gamma = 0.3, n_jobs=-1)
        newxgb_g3.fit(X_train_g3,y_train_g3)
        #predictedCV3 = cross_val_predict(newxgb_g3, X_train_g3, y_train_g3, cv=self.cv)
        
        score3i=-cross_val_score(newxgb_g3, X_train_g3, y_train_g3, scoring=self.metric2opt, cv=4)

        score_mse3i=score3i
            
        if self.metric2opt != 'neg_mean_squared_error':
            newxgb_g3 = XGBRegressor(max_depth = bestparams_g3.get('max_depth'), n_estimators = bestparams_g3.get('n_estimators'),gamma = 0.3, n_jobs=-1)
            score_mse3i=-cross_val_score(newxgb_g3, X_train_g3, y_train_g3, scoring='neg_mean_squared_error', cv=self.cv)
        
        print('Class 3 Training Data MSE: %2f .... STDev: %2f' %(np.mean(score_mse3i),np.std(score_mse3i)))
        print('Class 3 Training Data RMSE: %2f' %(np.sqrt(np.mean(score_mse3i))))
        
        ###########################
        
        self.trainmatrixgroup = [train_g0,train_g1,train_g2,train_g3]
        self.trainxgroup = [X_train_g0,X_train_g1,X_train_g3,X_train_g3]
        self.trainygroup = [y_train_g0,y_train_g1,y_train_g2,y_train_g3]
        self.regressormatrix = [newxgb_g0,newxgb_g1,newxgb_g2,newxgb_g3]
        self.scorematrix = [score0i,score1i,score2i,score3i]
        self.scoremsematrix = [score_mse0i,score_mse1i,score_mse2i,score_mse3i]
        #self.predictCVmatrix = [predictedCV0,predictedCV1,predictedCV2,predictedCV3]
        
        ####################
        
        ###CLASSIFY###

        dtrain = xgb.DMatrix(self.group_X_train, label=self.group_y_train)
        dtest = xgb.DMatrix(self.group_X_test, label=self.group_y_test)
        
        self.dtrain = dtrain
        self.dtest = dtest
        
        xgbclsfr = xgb.XGBClassifier(num_class = 5,eta = 0.6)

        parametersMLR = {'max_depth':self.maxdepth_range,'n_estimators':self.nestimators_range}
        xgb_classifier = GridSearchCV(xgbclsfr, parametersMLR)

        xgb_classifier.fit(self.group_X_train,self.group_y_train)
        bestparamsMLR = xgb_classifier.best_params_
        param = {
            'max_depth': bestparamsMLR.get('max_depth'),  # the maximum depth of each tree
            'n_estimators': bestparamsMLR.get('n_estimators'), #N_Estimators
            'eta': 0.6,  # the training step for each iteration
            'silent': 1,  # logging mode - quiet
            'objective': 'multi:softprob',  # error evaluation for multiclass training
            'num_class': 4}  # the number of classes that exist in this datset
        num_round = 40  # the number of training iterations
        
        bst = xgb.train(param, self.dtrain, num_round)
        preds = bst.predict(self.dtest)
        
        best_preds = np.asarray([np.argmax(line) for line in preds])
        clsfrprecisionscore = precision_score(self.group_y_test, best_preds, average='macro')
        
        print('----------')
        print('CLASSIFIER ACCURACY ON TEST DATA:')
        print('Classifier Accuracy on Test Data: ',clsfrprecisionscore*100,'%')
        
        self.xgb_classifier = xgb_classifier
        
        if mltargetplanet == 'RemTot':
            mainactiondata = self.remtot_nogroup_data_TEST
        elif mltargetplanet == 'RemTarg':
            mainactiondata = self.remtarg_nogroup_data_TEST
        else:
            mainactiondata = self.xi_nogroup_data_TEST
            
        mainactiondata["Group"] = best_preds
            
        ######    
            
        mainactiondata.sort_values('Group')
        
        #Sorting Them Again
        
        newtest_g0 = mainactiondata[mainactiondata["Group"] == 0]
        newy_test_g0 = newtest_g0[mltargetplanet]
        del newtest_g0[mltargetplanet]
        newX_test_g0 = newtest_g0

        ######
        newtest_g1 = mainactiondata[mainactiondata["Group"] == 1]
        newy_test_g1 = newtest_g1[mltargetplanet]
        del newtest_g1[mltargetplanet]
        newX_test_g1 = newtest_g1

        #######
        newtest_g2 = mainactiondata[mainactiondata["Group"] == 2]
        newy_test_g2 = newtest_g2[mltargetplanet]
        del newtest_g2[mltargetplanet]
        newX_test_g2 = newtest_g2

        #######
        newtest_g3 = mainactiondata[mainactiondata["Group"] == 3]
        newy_test_g3 = newtest_g3[mltargetplanet]
        del newtest_g3[mltargetplanet]
        newX_test_g3 = newtest_g3

        ###########################

        self.newtestmatrixgroup = [newtest_g0,newtest_g1,newtest_g2,newtest_g3]
        self.newtestxgroup = [newX_test_g0,newX_test_g1,newX_test_g2,newX_test_g3]
        self.newtestygroup = [newy_test_g0,newy_test_g1,newy_test_g2,newy_test_g3]

        frame = []
        for i in range(0,4):
            y_test_predsm = self.regressormatrix[i].predict(self.newtestxgroup[i])
            self.newtestxgroup[i][mltargetplanet] = y_test_predsm
            frame.append(self.newtestxgroup[i])

        result = pd.concat(frame)
        self.result = result

        y_test_pred = result[mltargetplanet]
        
        y_test_pred.sort_index(inplace=True)
        
        if mltargetplanet == 'RemTot':
        
            self.remtot_nogroup_y_test.sort_index(inplace=True)
            
            print('----------')
            print('TESTING DATA SCORES:')
            #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(self.xgb_y_test, y_test_pred)))
            print('MAE : ....%2f' %(metrics.mean_absolute_error(self.remtot_nogroup_y_test, y_test_pred)))
            print('MSE : ....%2f' %(metrics.mean_squared_error(self.remtot_nogroup_y_test, y_test_pred)))
            print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(self.remtot_nogroup_y_test, y_test_pred))))
            
            self.remtotmse = metrics.mean_squared_error(self.remtot_nogroup_y_test, y_test_pred)
            
            self.remtotypred = y_test_pred
            self.remtotyact = self.remtot_nogroup_y_test

            return y_test_pred
        
        elif mltargetplanet == 'RemTarg':
        
            self.remtarg_nogroup_y_test.sort_index(inplace=True)

            print('----------')
            print('TESTING DATA SCORES:')
            #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(self.xgb_y_test, y_test_pred)))
            print('MAE : ....%2f' %(metrics.mean_absolute_error(self.remtarg_nogroup_y_test, y_test_pred)))
            print('MSE : ....%2f' %(metrics.mean_squared_error(self.remtarg_nogroup_y_test, y_test_pred)))
            print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(self.remtarg_nogroup_y_test, y_test_pred))))
            
            self.remtargmse = metrics.mean_squared_error(self.remtarg_nogroup_y_test, y_test_pred)
            
            self.remtargypred = y_test_pred
            self.remtargyact = self.remtarg_nogroup_y_test
            
            return y_test_pred
        
        else:
        
            self.xi_nogroup_y_test.sort_index(inplace=True)

            print('----------')
            print('TESTING DATA SCORES:')
            #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(self.xgb_y_test, y_test_pred)))
            print('MAE : ....%2f' %(metrics.mean_absolute_error(self.xi_nogroup_y_test, y_test_pred)))
            print('MSE : ....%2f' %(metrics.mean_squared_error(self.xi_nogroup_y_test, y_test_pred)))
            print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(self.xi_nogroup_y_test, y_test_pred))))
            
            self.ximse = metrics.mean_squared_error(self.xi_nogroup_y_test, y_test_pred)
            
            self.xiypred = y_test_pred
            self.xiyact = self.xi_nogroup_y_test
            
            return y_test_pred
    
    
    #------ANALYSIS CODE-------#
    
    def analysis(self,selectedmodel):
        
        self.selectedmodel = selectedmodel
        
        if self.selectedmodel == 'XGBoost':
                    
            if (self.metric2opt == 'custom'):
                custom_scorer = metrics.make_scorer(self.custom_loss_func, greater_is_better =False)
                self.metric=custom_scorer
            print('                   Using the '+selectedmodel +' Model                   ')
            print('')
            print('++++++++++ MASS OF REMNANT NORMALIZED TO TOTAL MASS ++++++++++')
            print('')
            print('==================== Original Four Features ====================')
            print('GRID SEARCH:')
            best_params_remtot=self.xgbGS('remtot')
            
            print('----------')
            print('TRAINING DATA:')
            [score_remtot, mse_remtot, y_predCV_remtot]=self.xgbCV_scores(self.remtot_nogroup_X_train, self.remtot_nogroup_y_train, best_params_remtot)
            
            # To get scores in cross validation with Train data
            #print('Training Data %s: %2f .... STDev: %2f ' %(self.metric2opt, np.mean(score_remtot), np.std(score_remtot)))
            print('Training Data MSE: %2f .... STDev: %2f' %(np.mean(mse_remtot),np.std(mse_remtot)))
            print('Training Data RMSE: %2f' %(np.sqrt(np.mean(mse_remtot))))
            
            print('')
            self.outlierdetector(self.remtot_nogroup_X_train,self.remtot_nogroup_y_train,y_predCV_remtot,0.25)
            
            #self.outlierdetector(self.remtot_nogroup_X_train,self.remtot_nogroup_y_train,y_predCV_remtot,0.25)
        
            xgb_final_remtot = self.xgbtrainmodel(self.remtot_nogroup_X_train, self.remtot_nogroup_y_train, best_params_remtot)
            print('----------')
            print('TEST DATA:')
            y_test_pred_remtot = self.xgbtest_unseendata(xgb_final_remtot,self.remtot_nogroup_X_test,self.remtot_nogroup_y_test)
            
            print('')
            self.outlierdetector(self.remtot_nogroup_X_test,self.remtot_nogroup_y_test,y_test_pred_remtot,0.25)
            
            print('')
            print('')
            
            print('==================== With Feature Generation ====================')
            self.remtot_nogroup_full_X_train = self.createfeatures(self.remtot_nogroup_X_train)
            self.remtot_nogroup_full_X_test = self.createfeatures(self.remtot_nogroup_X_test)
            print('GRID SEARCH:')
            best_params_remtotfull=self.xgbGS('remtotfull')
            
            print('----------')
            print('TRAINING DATA:')
            [score_remtotfull, mse_remtotfull, y_predCV_remtotfull]=self.xgbCV_scores(self.remtot_nogroup_full_X_train, self.remtot_nogroup_y_train, best_params_remtotfull)
            

            # To get scores in cross validation with Train data
            #print('Training Data %s: %2f .... STDev: %2f ' %(self.metric2opt, np.mean(score_remtot), np.std(score_remtot)))
            print('Training Data MSE: %2f .... STDev: %2f' %(np.mean(mse_remtotfull),np.std(mse_remtotfull)))
            print('Training Data RMSE: %2f' %(np.sqrt(np.mean(mse_remtotfull))))
            
            print('')
            self.outlierdetector(self.remtot_nogroup_full_X_train,self.remtot_nogroup_y_train,y_predCV_remtotfull,0.25)
            
            xgb_final_remtotfull = self.xgbtrainmodel(self.remtot_nogroup_full_X_train, self.remtot_nogroup_y_train, best_params_remtotfull)
            print('----------')
            print('TEST DATA:')
            y_test_pred_remtotfull = self.xgbtest_unseendata(xgb_final_remtotfull,self.remtot_nogroup_full_X_test,self.remtot_nogroup_y_test)
            
            print('')
            self.outlierdetector(self.remtot_nogroup_full_X_test,self.remtot_nogroup_y_test,y_test_pred_remtotfull,0.25)
            
            print('')
            print('')
            print('++++++++++ MASS OF REMNANT NORMALIZED TO TARGET MASS ++++++++++')
            print('')
            print('==================== Original Four Features ====================')
            print('GRID SEARCH:')
            best_params_remtarg=self.xgbGS('remtarg')
            
            print('----------')
            print('TRAINING DATA:')
            [score_remtarg, mse_remtarg, y_predCV_remtarg]=self.xgbCV_scores(self.remtarg_nogroup_X_train, self.remtarg_nogroup_y_train, best_params_remtarg)
            
            # To get scores in cross validation with Train data
            #print('Training Data %s: %2f .... STDev: %2f ' %(self.metric2opt, np.mean(score_remtot), np.std(score_remtot)))
            print('Training Data MSE: %2f .... STDev: %2f' %(np.mean(mse_remtarg),np.std(mse_remtarg)))
            print('Training Data RMSE: %2f' %(np.sqrt(np.mean(mse_remtarg))))
            
            print('')
            self.outlierdetector(self.remtarg_nogroup_X_train,self.remtarg_nogroup_y_train,y_predCV_remtarg,0.25)
        
        
            xgb_final_remtarg = self.xgbtrainmodel(self.remtarg_nogroup_X_train, self.remtarg_nogroup_y_train, best_params_remtarg)
            print('----------')
            print('TEST DATA:')
            y_test_pred_remtarg = self.xgbtest_unseendata(xgb_final_remtarg,self.remtarg_nogroup_X_test,self.remtarg_nogroup_y_test)
            
            print('')
            self.outlierdetector(self.remtarg_nogroup_X_test,self.remtarg_nogroup_y_test,y_test_pred_remtarg,0.25)

            
            print('')
            print('')
            
            print('==================== With Feature Generation ====================')
            self.remtarg_nogroup_full_X_train = self.createfeatures(self.remtarg_nogroup_X_train)
            self.remtarg_nogroup_full_X_test = self.createfeatures(self.remtarg_nogroup_X_test)
            print('GRID SEARCH:')
            best_params_remtargfull=self.xgbGS('remtargfull')
            
            print('----------')
            print('TRAINING DATA:')
            [score_remtargfull, mse_remtargfull, y_predCV_remtargfull]=self.xgbCV_scores(self.remtarg_nogroup_full_X_train, self.remtarg_nogroup_y_train, best_params_remtargfull)
            
            # To get scores in cross validation with Train data
            #print('Training Data %s: %2f .... STDev: %2f ' %(self.metric2opt, np.mean(score_remtot), np.std(score_remtot)))
            print('Training Data MSE: %2f .... STDev: %2f' %(np.mean(mse_remtargfull),np.std(mse_remtargfull)))
            print('Training Data RMSE: %2f' %(np.sqrt(np.mean(mse_remtargfull))))
        
            print('')
            self.outlierdetector(self.remtarg_nogroup_full_X_train,self.remtarg_nogroup_y_train,y_predCV_remtargfull,0.25)
            
        
            xgb_final_remtargfull = self.xgbtrainmodel(self.remtarg_nogroup_full_X_train, self.remtarg_nogroup_y_train, best_params_remtargfull)
            print('----------')
            print('TEST DATA:')
            y_test_pred_remtargfull = self.xgbtest_unseendata(xgb_final_remtargfull,self.remtarg_nogroup_full_X_test,self.remtarg_nogroup_y_test) 
            
            print('')
            self.outlierdetector(self.remtarg_nogroup_full_X_test,self.remtarg_nogroup_y_test,y_test_pred_remtargfull,0.25)
            
            print('')
            print('')
            
            print('++++++++++ ACCRETION EFFICIENCY ++++++++++')
            print('')
            print('==================== Original Four Features ====================')
            print('GRID SEARCH:')
            best_params_xi=self.xgbGS('xi')
            
            print('----------')
            print('TRAINING DATA:')
            [score_xi, mse_xi, y_predCV_xi]=self.xgbCV_scores(self.xi_nogroup_X_train, self.xi_nogroup_y_train, best_params_xi)
            
            # To get scores in cross validation with Train data
            #print('Training Data %s: %2f .... STDev: %2f ' %(self.metric2opt, np.mean(score_remtot), np.std(score_remtot)))
            print('Training Data MSE: %2f .... STDev: %2f' %(np.mean(mse_xi),np.std(mse_xi)))
            print('Training Data RMSE: %2f' %(np.sqrt(np.mean(mse_xi))))
            
            print('')
            self.outlierdetector(self.xi_nogroup_X_train,self.xi_nogroup_y_train,y_predCV_xi,0.25)
        
            xgb_final_xi = self.xgbtrainmodel(self.xi_nogroup_X_train, self.xi_nogroup_y_train, best_params_xi)
            print('----------')
            print('TEST DATA:')
            y_test_pred_xi = self.xgbtest_unseendata(xgb_final_xi,self.xi_nogroup_X_test,self.xi_nogroup_y_test)
            
            print('')
            self.outlierdetector(self.xi_nogroup_X_test,self.xi_nogroup_y_test,y_test_pred_xi,0.25)
            
            print('')
            print('')
            
            print('==================== With Feature Generation ====================')
            self.xi_nogroup_full_X_train = self.createfeatures(self.xi_nogroup_X_train)
            self.xi_nogroup_full_X_test = self.createfeatures(self.xi_nogroup_X_test)
            print('GRID SEARCH:')
            best_params_xifull=self.xgbGS('xifull')
            
            print('----------')
            print('TRAINING DATA:')
            [score_xifull, mse_xifull, y_predCV_xifull]=self.xgbCV_scores(self.xi_nogroup_full_X_train, self.xi_nogroup_y_train, best_params_xifull)
            
            # To get scores in cross validation with Train data
            #print('Training Data %s: %2f .... STDev: %2f ' %(self.metric2opt, np.mean(score_remtot), np.std(score_remtot)))
            print('Training Data MSE: %2f .... STDev: %2f' %(np.mean(mse_xifull),np.std(mse_xifull)))
            print('Training Data RMSE: %2f' %(np.sqrt(np.mean(mse_xifull))))
            
            print('')
            self.outlierdetector(self.xi_nogroup_full_X_train,self.xi_nogroup_y_train,y_predCV_xifull,0.25)
            
        
            xgb_final_xifull = self.xgbtrainmodel(self.xi_nogroup_full_X_train, self.xi_nogroup_y_train, best_params_xifull)
            print('----------')
            print('TEST DATA:')
            y_test_pred_xifull = self.xgbtest_unseendata(xgb_final_xifull,self.xi_nogroup_full_X_test,self.xi_nogroup_y_test)
            
            print('')
            self.outlierdetector(self.xi_nogroup_full_X_test,self.xi_nogroup_y_test,y_test_pred_xifull,0.25)
            

            print('')
            print('')
            
            self.xgbremtotpred = y_test_pred_remtotfull
            self.xgbremtargpred = y_test_pred_remtargfull
            self.xgbxipred = y_test_pred_xifull
            
            scorearray = np.array([np.mean(mse_remtotfull),np.mean(mse_remtargfull),np.mean(mse_xifull)])
            regressorlist = [xgb_final_remtot,xgb_final_remtarg,xgb_final_xi]
            scorewinner = np.argmin(scorearray)
            
            if scorewinner == 0:
                print('BEST TRAIN MSE SCORE WAS: ',np.mean(mse_remtotfull),' FOR MACHINE LEARNING TARGET - MASS OF LARGEST REMNANT NORMALIZED TO TOTAL MASS')
            elif scorewinner == 1:
                print('BEST TRAIN MSE SCORE WAS: ',np.mean(mse_remtargfull),' FOR MACHINE LEARNING TARGET - MASS OF LARGEST REMNANT NORMALIZED TO TARGET MASS')
            else:
                print('BEST TRAIN MSE SCORE WAS: ',np.mean(mse_xifull),' FOR MACHINE LEARNING TARGET - ACCRETION EFFICIENCY')
        
            trace_remtotfull_train = go.Scatter(x=self.remtot_nogroup_y_train,y=y_predCV_remtotfull, name="MLR Norm. to Total M - Train Data",mode= 'markers',marker= dict(size= 3+3*self.remtot_nogroup_full_X_train['Impact Velocity'],line= dict(width=1), color= self.remtot_nogroup_full_X_train['Impact Angle'],colorscale='Reds',showscale=False))
            trace_remtotfull_test = go.Scatter(x=self.remtot_nogroup_y_test,y=y_test_pred_remtotfull, name="MLR Norm. to Total M - Test Data",mode= 'markers',marker= dict(size= 3+3*self.remtot_nogroup_full_X_test['Impact Velocity'],line= dict(width=1), color= self.remtot_nogroup_full_X_test['Impact Angle'],colorscale='Greens',showscale=False))
            
            trace_remtargfull_train = go.Scatter(x=self.remtarg_nogroup_y_train,y=y_predCV_remtargfull, name="MLR Norm. to Target M - Train Data",mode= 'markers',marker= dict(size= 3+3*self.remtarg_nogroup_full_X_train['Impact Velocity'],line= dict(width=1), color= self.remtarg_nogroup_full_X_train['Impact Angle'],colorscale='Blues',showscale=False))
            trace_remtargfull_test = go.Scatter(x=self.remtarg_nogroup_y_test,y=y_test_pred_remtargfull, name="MLR Norm. to Target M - Test Data",mode= 'markers',marker= dict(size= 3+3*self.remtarg_nogroup_full_X_test['Impact Velocity'],line= dict(width=1), color= self.remtarg_nogroup_full_X_test['Impact Angle'],colorscale='Blackbody',showscale=False))

            trace_xifull_train = go.Scatter(x=self.xi_nogroup_y_train,y=y_predCV_xifull, name="Xi - Train Data",mode= 'markers',marker= dict(size= 3+3*self.xi_nogroup_full_X_train['Impact Velocity'],line= dict(width=1), color= self.xi_nogroup_full_X_train['Impact Angle'],colorscale='Hot',showscale=False))
            trace_xifull_test = go.Scatter(x=self.xi_nogroup_y_test,y=y_test_pred_xifull, name="Xi - Test Data",mode= 'markers',marker= dict(size= 3+3*self.xi_nogroup_full_X_test['Impact Velocity'],line= dict(width=1), color= self.xi_nogroup_full_X_test['Impact Angle'],colorscale='Viridis',showscale=False))

            datasetplot = [trace_remtotfull_train,trace_remtotfull_test,trace_remtargfull_train,trace_remtargfull_test,trace_xifull_train,trace_xifull_test]
            #py.plot(datasetplot)
                    
            layout = go.Layout(title='Actual Vs Predicted Graphs',
                          xaxis = dict(title='Actual', zeroline = False),
                          yaxis = dict(title='Predicted',zeroline = False)
                         )
            fig = go.Figure(data=datasetplot, layout=layout)
            return(py.iplot(fig))
        
        elif self.selectedmodel == 'Nested':
                    
            if (self.metric2opt == 'custom'):
                custom_scorer = metrics.make_scorer(self.custom_loss_func, greater_is_better =False)
                self.metric=custom_scorer
                
            print('                   Using the '+selectedmodel +' Model                   ')
            print('')
            print('++++++++++ MASS OF REMNANT NORMALIZED TO TOTAL MASS ++++++++++')
            print('')
            print('==================== Original Four Features ====================')
             
            self.nest_main(self.remtot_wgroup_data_TRAIN,'RemTot')
            
            print('')
            print('')
            print('++++++++++ MASS OF REMNANT NORMALIZED TO TARGET MASS ++++++++++')
            print('')
            print('==================== Original Four Features ====================')
            
            self.nest_main(self.remtarg_wgroup_data_TRAIN,'RemTarg')
            
            print('')
            print('')
            print('++++++++++ ACCRETION EFFICIENCY ++++++++++')
            print('')
            print('==================== Original Four Features ====================')
            
            self.nest_main(self.xi_wgroup_data_TRAIN,'Xi')
            
            print('')
            print('')
            
            scorearray = np.array([self.remtotmse,self.remtargmse,self.ximse])
            scorewinner = np.argmin(scorearray)
            
            if scorewinner == 0:
                print('BEST TRAIN MSE SCORE WAS: ',self.remtotmse,' FOR MACHINE LEARNING TARGET - MASS OF LARGEST REMNANT NORMALIZED TO TOTAL MASS')
            elif scorewinner == 1:
                print('BEST TRAIN MSE SCORE WAS: ',self.remtargmse,' FOR MACHINE LEARNING TARGET - MASS OF LARGEST REMNANT NORMALIZED TO TARGET MASS')
            else:
                print('BEST TRAIN MSE SCORE WAS: ',self.ximse,' FOR MACHINE LEARNING TARGET - ACCRETION EFFICIENCY')
            
            
            trace_remtot_test = go.Scatter(x=self.remtotyact,y=self.remtotypred, name="MLR Norm. to Total M - Test Data",mode= 'markers',marker= dict(size= 3+3*self.result['Impact Velocity'],line= dict(width=1), color=self.result['Impact Angle'],colorscale='Greens',showscale=False))
            
            trace_remtarg_test = go.Scatter(x=self.remtargyact,y=self.remtargypred, name="MLR Norm. to Target M - Test Data",mode= 'markers',marker= dict(size= 3+3*self.result['Impact Velocity'],line= dict(width=1), color=self.result['Impact Angle'],colorscale='Blackbody',showscale=False))

            trace_xi_test = go.Scatter(x=self.xiyact,y=self.xiypred, name="Xi - Test Data",mode= 'markers',marker= dict(size= 3+3*self.result['Impact Velocity'],line= dict(width=1), color=self.result['Impact Angle'],colorscale='Viridis',showscale=False))

            datasetplot = [trace_remtot_test,trace_remtarg_test,trace_xi_test]
            #py.plot(datasetplot)
                    
            layout = go.Layout(title='Actual Vs Predicted Graphs',
                          xaxis = dict(title='Actual', zeroline = False),
                          yaxis = dict(title='Predicted',zeroline = False)
                         )
            fig = go.Figure(data=datasetplot, layout=layout)
            return(py.iplot(fig))
            
            #return y_test_pred_remtotfull
            
        
        elif self.selectedmodel == 'Ensemble':
            
            xgb_remtot_pred = self.xgbremtotpred.sort_index(inplace=True)
            xgb_remtarg_pred = self.xgbremtargpred.sort_index(inplace=True)
            xgb_xi_pred = self.xgbxipred.sort_index(inplace=True)
            
            nested_remtot_pred = self.remtotypred.sort_index(inplace=True)
            nested_remtarg_pred = self.remtargypred.sort_index(inplace=True)
            nested_xi_pred = self.xiypred.sort_index(inplace=True)
            
            actual_remtot_test = self.remtot_nogroup_y_test.sort_index(inplace=True)
            actual_remtarg_test = self.remtarg_nogroup_y_test.sort_index(inplace=True)
            actual_xi_test = self.xi_nogroup_y_test.sort_index(inplace=True)
            
            aggregate_remtot_pred = (xgb_remtot_pred+nested_remtot_pred)/2
            aggregate_remtarg_pred = (xgb_remtarg_pred+nested_remtarg_pred)/2
            aggregate_xi_pred = (xgb_xi_pred+nested_xi_pred)/2
            
            
            print('                   Using the '+selectedmodel +' Model                   ')
            print('')
            print('++++++++++ MASS OF REMNANT NORMALIZED TO TOTAL MASS ++++++++++')
            print('')
            
            
            print('TESTING DATA SCORES:')
            #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(self.xgb_y_test, y_test_pred)))
            print('MAE : ....%2f' %(metrics.mean_absolute_error(actual_remtot_test, aggregate_remtot_pred)))
            print('MSE : ....%2f' %(metrics.mean_squared_error(actual_remtot_test, aggregate_remtot_pred)))
            print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(actual_remtot_test, aggregate_remtot_pred))))
            
            
            agremtotmse = metrics.mean_squared_error(actual_remtot_test, aggregate_remtot_pred)
            
            print('')
            print('++++++++++ MASS OF REMNANT NORMALIZED TO TARGET MASS ++++++++++')
            print('')
            
            
            print('TESTING DATA SCORES:')
            #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(self.xgb_y_test, y_test_pred)))
            print('MAE : ....%2f' %(metrics.mean_absolute_error(actual_remtarg_test, aggregate_remtarg_pred)))
            print('MSE : ....%2f' %(metrics.mean_squared_error(actual_remtarg_test, aggregate_remtarg_pred)))
            print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(actual_remtarg_test, aggregate_remtarg_pred))))
            
            agremtargmse = metrics.mean_squared_error(actual_remtarg_test, aggregate_remtarg_pred)
            
            print('')
            print('++++++++++ ACCRETION EFFICIENCY ++++++++++')
            print('')
            
            
            print('TESTING DATA SCORES:')
            #print('%s : .....%2f' %(self.metric2opt, self.custom_loss_func(self.xgb_y_test, y_test_pred)))
            print('MAE : ....%2f' %(metrics.mean_absolute_error(actual_xi_test, aggregate_xi_pred)))
            print('MSE : ....%2f' %(metrics.mean_squared_error(actual_xi_test, aggregate_xi_pred)))
            print('RMSE : ....%2f' %(np.sqrt(metrics.mean_squared_error(actual_xi_test, aggregate_xi_pred))))
            
            agximse = metrics.mean_squared_error(actual_xi_test, aggregate_xi_pred)
            
            print('')
            print('')
            
            scorearray = np.array([agremtotmse,agremtargmse,agximse])
            scorewinner = np.argmin(scorearray)
            
            if scorewinner == 0:
                print('BEST TRAIN MSE SCORE WAS: ',agremtotmse,' FOR MACHINE LEARNING TARGET - MASS OF LARGEST REMNANT NORMALIZED TO TOTAL MASS')
            elif scorewinner == 1:
                print('BEST TRAIN MSE SCORE WAS: ',agremtargmse,' FOR MACHINE LEARNING TARGET - MASS OF LARGEST REMNANT NORMALIZED TO TARGET MASS')
            else:
                print('BEST TRAIN MSE SCORE WAS: ',agximse,' FOR MACHINE LEARNING TARGET - ACCRETION EFFICIENCY')
            
            
            trace_remtot_ag = go.Scatter(x=actual_remtot_test,y=aggregate_remtot_pred, name="MLR Norm. to Total M - Test Data",mode= 'markers',marker= dict(size= 3+3*self.result['Impact Velocity'],line= dict(width=1), color=self.result['Impact Angle'],colorscale='Greens',showscale=False))
            
            trace_remtarg_ag = go.Scatter(x=actual_remtarg_test,y=aggregate_remtarg_pred, name="MLR Norm. to Target M - Test Data",mode= 'markers',marker= dict(size= 3+3*self.result['Impact Velocity'],line= dict(width=1), color=self.result['Impact Angle'],colorscale='Blackbody',showscale=False))

            trace_xi_ag = go.Scatter(x=actual_xi_test,y=aggregate_xi_pred, name="Xi - Test Data",mode= 'markers',marker= dict(size= 3+3*self.result['Impact Velocity'],line= dict(width=1), color=self.result['Impact Angle'],colorscale='Viridis',showscale=False))

            datasetplot = [trace_remtot_ag,trace_remtarg_ag,trace_xi_ag]
            #py.plot(datasetplot)
                    
            layout = go.Layout(title='Actual Vs Predicted Graphs',
                          xaxis = dict(title='Actual', zeroline = False),
                          yaxis = dict(title='Predicted',zeroline = False)
                         )
            fig = go.Figure(data=datasetplot, layout=layout)
            return(py.iplot(fig))
            
        
        else:
            print("The selected model is invalid, please select either of the following:")
            print("1. XGBoost")
            print("2. Nested")
            print("3. Ensemble")        

In [9]:
alldatatp = pd.read_csv('collisions.csv') #Importing Data
run = MLCollisions(dataframe=alldatatp, metric2opt='neg_mean_squared_error', FI=True)
run.analysis('XGBoost')

                   Using the XGBoost Model                   

++++++++++ MASS OF REMNANT NORMALIZED TO TOTAL MASS ++++++++++

==================== Original Four Features ====================
GRID SEARCH:
Performing GridSearchCV with Metric: neg_mean_squared_error
Best Parameters: {'max_depth': 9, 'n_estimators': 65}
Best Metric Score: 0.00892347478082458
----------
TRAINING DATA:
Training Data MSE: 0.008924 .... STDev: 0.001470
Training Data RMSE: 0.094466

Number of samples predicted poorly: 13
MSE of poorly-predicted samples:  0.128418
MSE of well-predicted samples:  0.006874
----------
NON-ZERO FEATURE IMPORTANCES:
Target Mass: .... 0.016529
Impactor Mass/Target Mass: .... 0.214876
Impact Velocity: .... 0.471074
Impact Angle: .... 0.297521
----------
TEST DATA:
Test Data Scores:
MAE : ....0.065055
MSE : ....0.006762
RMSE : ....0.082233

Number of samples predicted poorly: 3
MSE of poorly-predicted samples:  0.106890
MSE of well-predicted samples:  0.005584


==================== Wi

In [10]:
alldatatp = pd.read_csv('collisions.csv') #Importing Data
run = MLCollisions(dataframe=alldatatp, metric2opt='neg_mean_squared_error', FI=True)
run.analysis('Nested')

                   Using the Nested Model                   

++++++++++ MASS OF REMNANT NORMALIZED TO TOTAL MASS ++++++++++

==================== Original Four Features ====================
CLASS-SPECIFIC REGRESSOR TRAINING:
Class 0 Training Data MSE: 0.012270 .... STDev: 0.003140
Class 0 Training Data RMSE: 0.110768

Class 1 Training Data MSE: 0.004720 .... STDev: 0.006247
Class 1 Training Data RMSE: 0.068700

Class 2 Training Data MSE: 0.004289 .... STDev: 0.000798
Class 2 Training Data RMSE: 0.065489

Class 3 Training Data MSE: 0.009411 .... STDev: 0.002139
Class 3 Training Data RMSE: 0.097012
----------
CLASSIFIER ACCURACY ON TEST DATA:
Classifier Accuracy on Test Data:  96.7758367883507 %
----------
TESTING DATA SCORES:
MAE : ....0.063645
MSE : ....0.008489
RMSE : ....0.092134


++++++++++ MASS OF REMNANT NORMALIZED TO TARGET MASS ++++++++++

==================== Original Four Features ====================
CLASS-SPECIFIC REGRESSOR TRAINING:
Class 0 Training Data MSE: 0.014973 ...